<a href="https://colab.research.google.com/github/ptx-pluto/colab-notebooks/blob/master/rnw_sympy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import sympy
sympy.init_printing()

In [125]:
def rot_x(rad):
  c = sympy.cos(rad)
  s = sympy.sin(rad)
  return sympy.Matrix([[1,0, 0],
                       [0,c,-s],
                       [0,s, c]])

def rot_y(rad):
  c = sympy.cos(rad)
  s = sympy.sin(rad)  
  return sympy.Matrix([[ c,0,s],
                       [ 0,1,0],
                       [-s,0,c]])

def rot_z(rad):
  c = sympy.cos(rad)
  s = sympy.sin(rad)
  return sympy.Matrix([[c,-s,0],
                       [s, c,0],
                       [0, 0,1]])

In [126]:
cone = {
    'm': sympy.symbols('m'),
    'g': sympy.symbols('g'),
    'R': sympy.symbols('r'),
    'xCM': sympy.symbols('xCM'),
    'zCM': sympy.symbols('zCM'),
    'xCP': sympy.symbols('xCP'),
    'zCP': sympy.symbols('zCP'),
}

### Coordinate System

5 Generalized Coordiates: $q_1, q_2, q_4, q_5, q_6$

$q_1 = x$

$q_2 = y$

$q_3 = \psi$ (precession)

$q_4 = \theta$ (nutation)

$q_5 = \phi$ (spin)

In [127]:
t = sympy.symbols('t')

q1 = sympy.Function('q_1')(t)
q2 = sympy.Function('q_2')(t)
q3 = sympy.Function('q_3')(t)
q4 = sympy.Function('q_4')(t)
q5 = sympy.Function('q_5')(t)

dq1 = sympy.Function('\dot{q}_1')(t)
dq2 = sympy.Function('\dot{q}_2')(t)
dq3 = sympy.Function('\dot{q}_3')(t)
dq4 = sympy.Function('\dot{q}_4')(t)
dq5 = sympy.Function('\dot{q}_5')(t)

psi   = q3
theta = q4
phi   = q5

psi_dot   = dq3
theta_dot = dq4
phi_dot   = dq5

ddq1, ddq2, ddq3, ddq4, ddq5 = sympy.symbols('\ddot{q}_1 \ddot{q}_2 \ddot{q}_3 \ddot{q}_4 \ddpt{q}_5')

q = (q1, q2, q3, q4, q5)
dq = (dq1, dq2, dq3, dq4, dq5)
ddq = (ddq1, ddq2, ddq3, ddq4, ddq5)

sub_rules = [(q1.diff(),dq1),(dq1.diff(),ddq1),
             (q2.diff(),dq2),(dq2.diff(),ddq2),
             (q3.diff(),dq3),(dq3.diff(),ddq3),
             (q4.diff(),dq4),(dq4.diff(),ddq4),
             (q5.diff(),dq5),(dq5.diff(),ddq5)]

### Rigid Body Transformation

$\vec{P}_{W} = R_{WB} \vec{P}_B + T_{WB}$

$R_{WB} = R_z(\psi) R_y(\theta) R_z(\phi)$

$T_{WB} = (\ q_1, q_2, \mathbf{r} sin(q_5) \ )^T$

In [128]:
R_wb = rot_z(psi) * rot_y(theta) * rot_z(phi)
T_wb = sympy.Matrix([ q1, q2, cone['R'] * sympy.sin(theta)])
R_wb

⎡-sin(q₃(t))⋅sin(q₅(t)) + cos(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t))  -sin(q₃(t))⋅cos(q₅
⎢                                                                             
⎢sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) + sin(q₅(t))⋅cos(q₃(t))   -sin(q₃(t))⋅sin(q₅
⎢                                                                             
⎣                 -sin(q₄(t))⋅cos(q₅(t))                                      

(t)) - sin(q₅(t))⋅cos(q₃(t))⋅cos(q₄(t))  sin(q₄(t))⋅cos(q₃(t))⎤
                                                              ⎥
(t))⋅cos(q₄(t)) + cos(q₃(t))⋅cos(q₅(t))  sin(q₃(t))⋅sin(q₄(t))⎥
                                                              ⎥
sin(q₄(t))⋅sin(q₅(t))                         cos(q₄(t))      ⎦

In [129]:
CoM_b = sympy.Matrix([ cone['xCM'], 0, cone['zCM'] ])
CoM_b

⎡xCM⎤
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎣zCM⎦

In [130]:
CoM_w = R_wb * CoM_b + T_wb
CoM_w

⎡xCM⋅(-sin(q₃(t))⋅sin(q₅(t)) + cos(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t))) + zCM⋅sin(q₄(
⎢                                                                             
⎢xCM⋅(sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) + sin(q₅(t))⋅cos(q₃(t))) + zCM⋅sin(q₃(t
⎢                                                                             
⎣                     r⋅sin(q₄(t)) - xCM⋅sin(q₄(t))⋅cos(q₅(t)) + zCM⋅cos(q₄(t)

t))⋅cos(q₃(t)) + q₁(t)⎤
                      ⎥
))⋅sin(q₄(t)) + q₂(t) ⎥
                      ⎥
)                     ⎦

In [131]:
CoM_w_dot = sympy.simplify(CoM_w.diff(t).subs(sub_rules))
CoM_w_dot

⎡-xCM⋅(\dot{q}₃(t)⋅sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) + \dot{q}₃(t)⋅sin(q₅(t))⋅c
⎢                                                                             
⎢-xCM⋅(\dot{q}₃(t)⋅sin(q₃(t))⋅sin(q₅(t)) - \dot{q}₃(t)⋅cos(q₃(t))⋅cos(q₄(t))⋅c
⎢                                                                             
⎣                                                                             

os(q₃(t)) + \dot{q}₄(t)⋅sin(q₄(t))⋅cos(q₃(t))⋅cos(q₅(t)) + \dot{q}₅(t)⋅sin(q₃(
                                                                              
os(q₅(t)) + \dot{q}₄(t)⋅sin(q₃(t))⋅sin(q₄(t))⋅cos(q₅(t)) + \dot{q}₅(t)⋅sin(q₃(
                                                                              
            r⋅\dot{q}₄(t)⋅cos(q₄(t)) - xCM⋅\dot{q}₄(t)⋅cos(q₄(t))⋅cos(q₅(t)) +

t))⋅cos(q₅(t)) + \dot{q}₅(t)⋅sin(q₅(t))⋅cos(q₃(t))⋅cos(q₄(t))) - zCM⋅\dot{q}₃(
                                                                              
t))⋅sin(q₅(t))⋅cos(q₄(t)) - \dot{q}₅(t)⋅cos(q₃(t))

In [132]:
T = sympy.S.Half * cone['m'] * (CoM_w_dot.dot(CoM_w_dot))
V = cone['m'] * cone['g'] * CoM_w[2,0]
L = T-V
L = sympy.simplify(L)

### Equations of Motion from Lagarangian

$$
\frac{\partial}{dt}\frac{\partial L}{\partial \dot{q}_i} - \frac{\partial L}{\partial q_i} = 0
$$

In [133]:
eoms = sympy.Matrix([
      (L.diff(dq1).diff(t)-L.diff(q1)).subs(sub_rules),
      (L.diff(dq2).diff(t)-L.diff(q2)).subs(sub_rules),
      (L.diff(dq3).diff(t)-L.diff(q3)).subs(sub_rules),
      (L.diff(dq4).diff(t)-L.diff(q4)).subs(sub_rules),
      (L.diff(dq5).diff(t)-L.diff(q5)).subs(sub_rules)
])

In [134]:
def linear_decomp(expr,syms):
  def calc_term(i):
    return expr.subs([(syms[j],sympy.S.One) if i==j else (syms[j],0) for j in range(len(syms))])
  return [ calc_term(i) for i in range(len(syms))]

### Rewrite Equations of Motion in Matrix Form

$$
M(q)\ddot{q} + h(q,\dot{q}) = 0
$$


In [135]:
h = eoms.subs([(ddq1,0),(ddq2,0),(ddq3,0),(ddq4,0),(ddq5,0)])
Mddq = eoms-h

M = sympy.Matrix([linear_decomp(Mddq[i,0],ddq) for i in range(5)])

In [136]:
M = sympy.simplify(M)
h = sympy.simplify(h)

### Nonholonomic Constrain (Rolling Constrain)

$$
\dot{X}=R(\dot{\psi} \cos \theta+\dot{\phi}) \cos \psi-R \dot{\theta} \sin \psi \sin \theta
$$

$$
\dot{Y}=R(\dot{\psi} \cos \theta+\dot{\phi}) \sin \psi+R \dot{\theta} \cos \psi \sin \theta
$$

$$
\text{Paffian Form: } A_{nh}(q)\dot{q}=0
$$

In [137]:
eq1 = cone['R'] * (psi_dot * sympy.cos(theta) + phi_dot) * sympy.cos(psi) - \
      cone['R'] * theta_dot * sympy.sin(psi) * sympy.sin(theta) - dq1

eq2 = cone['R'] * (psi_dot * sympy.cos(theta) + phi_dot) * sympy.sin(psi) + \
      cone['R'] * theta_dot * sympy.cos(psi) * sympy.sin(theta) - dq2

A_nh = sympy.Matrix([linear_decomp(eq,dq) for eq in [eq1,eq2]])
A_nh

⎡-1  0   r⋅cos(q₃(t))⋅cos(q₄(t))  -r⋅sin(q₃(t))⋅sin(q₄(t))  r⋅cos(q₃(t))⎤
⎢                                                                       ⎥
⎣0   -1  r⋅sin(q₃(t))⋅cos(q₄(t))  r⋅sin(q₄(t))⋅cos(q₃(t))   r⋅sin(q₃(t))⎦

### Holonomic Constrain (Fixed Control Point)

$\dot{X}_{CP} = 0$

In [138]:
CP_b = sympy.Matrix([ cone['zCP'], 0, cone['zCP'] ])
CP_w = R_wb * CP_b + T_wb
CP_dot_w = CP_w.diff(t).subs(sub_rules)
A_cp = sympy.Matrix([linear_decomp(eq,dq) for eq in CP_dot_w])

In [139]:
#sympy.simplify(A_cp[:,2:5].det())

### Paffian Constrain Matrix

$\mathbf{A}\dot{q} = 0$

$\dot{\mathbf{A}} \ddot{q} = 0$

In [140]:
A = sympy.Matrix([-A_nh, A_cp])
A = sympy.simplify(A)
A

⎡1  0                                 -r⋅cos(q₃(t))⋅cos(q₄(t))                
⎢                                                                             
⎢0  1                                 -r⋅sin(q₃(t))⋅cos(q₄(t))                
⎢                                                                             
⎢1  0  -zCP⋅(sin(q₃(t))⋅sin(q₄(t)) + sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) + sin(q₅
⎢                                                                             
⎢0  1  zCP⋅(-sin(q₃(t))⋅sin(q₅(t)) + sin(q₄(t))⋅cos(q₃(t)) + cos(q₃(t))⋅cos(q₄
⎢                                                                             
⎣0  0                                             0                           

                                   r⋅sin(q₃(t))⋅sin(q₄(t))                    
                                                                              
                                  -r⋅sin(q₄(t))⋅cos(q₃(t))                    
                                                   

In [141]:
dAdt = A.diff(t).subs(sub_rules)
dAdt = sympy.simplify(dAdt)

### Equation of Motion


$\left[\begin{array}{cc}-\mathbf{A} & 0 \\ \mathbf{M} & -\mathbf{A}^{T}\end{array}\right]\left[\begin{array}{l}\ddot{\mathbf{q}} \\ \boldsymbol{\lambda}\end{array}\right]=\left[\begin{array}{c}\dot{\mathbf{A}} \dot{\mathbf{q}} \\ -\mathbf{h}\end{array}\right]$

In [142]:
M_left = sympy.Matrix([[-A,sympy.zeros(5)],[M, -A.T]])
M_right = sympy.Matrix([[dAdt*sympy.Matrix(dq)],[-h]])

In [143]:
mq = sympy.MatrixSymbol('q',5,1)
mdq = sympy.MatrixSymbol('dq',5,1)

sub_rule_out = [(q1,mq[0,0]),(dq1,mdq[0,0]),
                (q2,mq[1,0]),(dq2,mdq[1,0]),
                (q3,mq[2,0]),(dq3,mdq[2,0]),
                (q4,mq[3,0]),(dq4,mdq[3,0]),
                (q5,mq[4,0]),(dq5,mdq[4,0])]

e_M_left = M_left.subs(sub_rule_out)
e_M_right = M_right.subs(sub_rule_out)

In [144]:
e_M_left.free_symbols, e_M_right.free_symbols

In [145]:
f_M_left = sympy.lambdify([mq,cone['R'],cone['m'],cone['g'],cone['xCM'],cone['zCM'],cone['xCP'],cone['zCP']],e_M_left)
f_M_right = sympy.lambdify([mq,mdq,cone['R'],cone['m'],cone['g'],cone['xCM'],cone['zCM'],cone['xCP'],cone['zCP']],e_M_right)

In [146]:
outputs = {
    'M_left': f_M_left,
    'M_right': f_M_right
}
import dill
dill.settings['recurse'] = True
dill.dump(outputs,open("rnw", "wb"))

In [147]:
rnw_model = dill.load(open("rnw", "rb"))
import numpy as np
fake_q = np.array([[0,0,0,0,0]]).T
fake_dq = np.array([[1,1,1,1,1]]).T
rnw_model['M_left'](fake_q,1,1,9.8,1,1,2,3)
rnw_model['M_right'](fake_q,fake_dq,1,1,9.8,1,1,2,3)

func2 = rnw_model['M_right']
??func2